In [8]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import re
from datetime import datetime
import numpy as np
from configparser import ConfigParser
import unidecode

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

# get all lidl offer url

In [9]:
def get_all_link_lidl():
    
    offer_page = 'https://www.lidl.hu/ajanlataink'
    
    page = urllib.request.urlopen(offer_page)
    soup = bs(page)
    
    divs_body = soup.body.findAll('div', {'class' : ['tabnavaccordion__content']})

    all_link = []
    for div in divs_body:
        for a in div.find_all('a', href=True):
            url = 'https://www.lidl.hu' + a['href']
            #print(url)
            all_link.append(url)
            
    return all_link

In [10]:
all_link = get_all_link_lidl()

In [12]:
def get_all_offer_lidl(all_link):
    
    def find_nth_occurrence(string, char, occurrence):

        val = -1
        for i in range(0, occurrence):
            val = string.find(char, val + 1)
        return val

    all_items = []
    
    counter = 1

    for url in all_link:

        print(f'crawl url: {url} done {counter} from {len(all_link)}')
        counter = counter + 1

        page = urllib.request.urlopen(url)
        soup = bs(page)

        divs = soup.body.findAll('div', {'class' : ['nuc-a-flex-item']})

        for div in divs:
            articles = div.findAll('article', {'class' : 'ret-o-card'})
            for article in articles:

                item_dict = {}

                #print(article['data-id'])
                item_dict['itemId'] = article['data-id']

                #print(article['data-name'])
                brand = article.find('p', {'class' : 'ret-o-card__content'})

                if brand != None:
                    #print(brand.get_text().strip())
                    item_dict['itemName'] = article['data-name'] + ' - ' + brand.get_text().strip()
                else:
                    item_dict['itemName'] = article['data-name']
                    
                item_dict['itemCleanName'] = unidecode.unidecode(item_dict['itemName']).lower()

                #print(article['data-price'])
                item_dict['price'] = article['data-price']
                
                measure = article.find('div', {'class' : 'lidl-m-pricebox__basic-quantity'})
                
                if measure != None:
                    item_dict['measure'] = measure.get_text()
                else:
                    item_dict['measure'] = np.nan

                #print(article['data-list'])

                sales_from_pattern = r'(?P<group_1>[\d]{2}.[\d]{2})'

                if (article['data-list'] != None) and (re.search(sales_from_pattern, article['data-list']) != None):

                    sales_data = article['data-list']
                    item_dict['salesStart'] = str(datetime.now().year) + '.' + re.search(sales_from_pattern,
                                                                                          article['data-list'])[0]
                else:
                    item_dict['salesStart'] = np.nan

                cut_url = url[find_nth_occurrence(url, '/', 4)+1:]

                item_dict['source'] = cut_url[:cut_url.find('/')]
                item_dict['runDate'] = datetime.now().strftime('%Y.%m.%d-%H:%M:%S')
                item_dict['shopName'] = 'lidl'

                if len(item_dict) > 0:
                    all_items.append(item_dict)

                #print('-----')

    df = pd.DataFrame(all_items)
    
    return df

In [13]:
df = get_all_offer_lidl(all_link)

crawl url: https://www.lidl.hu/c/kerti-es-szabadteri-ajanlataink/c3500/w1 done 1 from 18
crawl url: https://www.lidl.hu/c/konyhai-ajanlataink/c3501/w1 done 2 from 18
crawl url: https://www.lidl.hu/c/akcioink-06-23-csutortoktol/c3505/w1 done 3 from 18
crawl url: https://www.lidl.hu/c/zoldseg-gyumolcs-akcioink-06-23-csutortoktol/c3506/w1 done 4 from 18
crawl url: https://www.lidl.hu/c/virag-ajanlataink-06-23-csutortoktol/c3507/w1 done 5 from 18
crawl url: https://www.lidl.hu/c/jegkrem-ajanlataink/c3508/w1 done 6 from 18
crawl url: https://www.lidl.hu/c/ismerje-meg-andaluzia-izvilagat/c3509/w1 done 7 from 18
crawl url: https://www.lidl.hu/c/barkacs-ajanlataink/c3510/w1 done 8 from 18
crawl url: https://www.lidl.hu/c/gyermek-textil-es-jatek-ajanlataink/c3511/w1 done 9 from 18
crawl url: https://www.lidl.hu/c/szuper-hetvege-06-25-szombattol/c3512/w1 done 10 from 18
crawl url: https://www.lidl.hu/c/akcioink-06-23-csutortoktol/c3505/w2 done 11 from 18
crawl url: https://www.lidl.hu/c/jegkrem-

In [7]:
df

,itemId,itemName,itemCleanName,price,measure,salesStart,source,runDate,shopName
0,384563,Benzinmotoros fűkasza - PARKSIDE,benzinmotoros fukasza - parkside,47999,/db,NaN,kerti-es-szabadteri-ajanlataink,2022.06.23-20:54:15,lidl
1,384536,Akkus fű- és sövényvágó - PARKSIDE,akkus fu- es sovenyvago - parkside,7999,/db,NaN,kerti-es-szabadteri-ajanlataink,2022.06.23-20:54:15,lidl
2,384572,Benzinmotoros fűnyíró - PARKSIDE,benzinmotoros funyiro - parkside,58888,/db,NaN,kerti-es-szabadteri-ajanlataink,2022.06.23-20:54:15,lidl
3,382693,Szélforgó - LIVARNO HOME,szelforgo - livarno home,1599,/db,NaN,kerti-es-szabadteri-ajanlataink,2022.06.23-20:54:15,lidl
4,376271,Prémium akkus sövénynyíró - PARKSIDE PERFORMANCE,premium akkus sovenynyiro - parkside performance,34999,/db,NaN,kerti-es-szabadteri-ajanlataink,2022.06.23-20:54:15,lidl
...,...,...,...,...,...,...,...,...,...
304,83100,Kapor,kapor,329,"25 g / csomag, 1 kg = 13160 Ft",2022.06.27,akcioink-06-27-hetfotol,2022.06.23-20:54:28,lidl
305,6401703,"Friss, szeletelt sertéskaraj - HÚSFARM","friss, szeletelt serteskaraj - husfarm",879,"400 g, 1 kg = 2198 Ft",2022.06.27,akcioink-06-27-hetfotol,2022.06.23-20:54:28,lidl
306,7732,Friss kacsacomb és bőrös kacsamellfilé csomag ...,friss kacsacomb es boros kacsamellfile csomag ...,2999,/kg,2022.06.27,akcioink-06-27-hetfotol,2022.06.23-20:54:28,lidl
307,6409711,Bismarck fánk,bismarck fank,189,"80 g, 1 kg = 2363 Ft",2022.06.27,akcioink-06-27-hetfotol,2022.06.23-20:54:28,lidl


In [14]:
df.to_excel('lidl_result.xlsx', index=False)

# backup

In [40]:
all_items = []

divs = soup.body.findAll('div', {'class' : ['nuc-a-flex-item']})

for div in divs:
    articles = div.findAll('article', {'class' : 'ret-o-card'})
    for article in articles:
            
            item_dict = {}
            
            #print(article['data-id'])
            item_dict['item_id'] = article['data-id']
            
            #print(article['data-name'])
            brand = article.find('p', {'class' : 'ret-o-card__content'})
            
            if brand != None:
                #print(brand.get_text().strip())
                item_dict['item_name'] = article['data-name'] + ' - ' + brand.get_text().strip()
            else:
                item_dict['item_name'] = article['data-name']
            
            #print(article['data-price'])
            item_dict['price'] = article['data-price']
            
            item_dict['measure'] = np.nan
            
            #print(article['data-list'])
            
            sales_from_pattern = r'(?P<group_1>[\d]{2}.[\d]{2})'
            
            if (article['data-list'] != None) and (re.search(sales_from_pattern, article['data-list']) != None):
            
                sales_data = article['data-list']
                item_dict['sales_start'] = str(datetime.now().year) + '.' + re.search(sales_from_pattern,
                                                                                      article['data-list'])[0]
            else:
                item_dict['sales_start'] = np.nan
                
            cut_url = url[find_nth_occurrence(url, '/', 4)+1:]
            
            item_dict['source'] = cut_url[:cut_url.find('/')]
            item_dict['run_date'] = datetime.now().strftime('%Y.%m.%d-%H:%M:%S')
            
            if len(item_dict) > 0:
                all_items.append(item_dict)
            
            #print('-----')

df = pd.DataFrame(all_items)

In [41]:
df

,item_id,item_name,price,measure,sales_start,source,run_date
0,160830,Jégkrém - GELATELLI,999,NaN,NaN,jegkrem-ajanlataink,2022.06.22-20:25:04
1,7304378,Málnás és vaníliaízű kevert jégkrém - GELATELLI,999,NaN,NaN,jegkrem-ajanlataink,2022.06.22-20:25:04
2,133450,Vanília-karamell-mogyoró ízű jégkrém - BALLINO,579,NaN,NaN,jegkrem-ajanlataink,2022.06.22-20:25:04
3,161350,Csavart jégkrém - BALLINO,1099,NaN,NaN,jegkrem-ajanlataink,2022.06.22-20:25:04
4,6804344,Sárgadinnyés jégkrém - GELATELLI,699,NaN,NaN,jegkrem-ajanlataink,2022.06.22-20:25:04
5,153333,Szendvics jégkrém - BALLINO,169,NaN,NaN,jegkrem-ajanlataink,2022.06.22-20:25:04
6,134227,Vaníliás-gyümölcsös jégkrém - BALLINO,179,NaN,NaN,jegkrem-ajanlataink,2022.06.22-20:25:04
7,6800157,Jégkrémtorta - BALLINO,1699,NaN,NaN,jegkrem-ajanlataink,2022.06.22-20:25:04
8,7303395,Fekete tölcséres jégkrém - GELATELLI,899,NaN,NaN,jegkrem-ajanlataink,2022.06.22-20:25:04
9,136791,Pálcikás jégkrém rágógumival - GELATELLI,899,NaN,NaN,jegkrem-ajanlataink,2022.06.22-20:25:04
